In [1]:
import os
import numpy as np
import utils

CMD=["perl", "../english_testbed/data/2015/evaluate/correlation-noconfidence.pl"]
class Opts:
    verbose=False
    filter_test=".*"
    
opts=Opts()

In [2]:
TRAIN_2015_DATADIR="english_testbed/analysis/2015/training/results/"
TEST_2015_DATADIR="english_testbed/analysis/2015/testing/results"
TEST_2015_DIR="english_testbed/data/2015/test/"
TRAIN_2015_NAMEFILES=[
  "STS.2012.test.input.MSRpar.txt",
  "STS.2012.test.input.MSRvid.txt",
  "STS.2012.test.input.SMTeuroparl.txt",
  "STS.2012.test.input.surprise.OnWN.txt",
  "STS.2012.test.input.surprise.SMTnews.txt",
  "STS.2012.train.input.MSRpar.txt",
  "STS.2012.train.input.MSRvid.txt",
  "STS.2012.train.input.SMTeuroparl.txt",
  "STS.2013.test.input.FNWN.txt",
  "STS.2013.test.input.headlines.txt",
  "STS.2013.test.input.OnWN.txt",
  "STS.2013.test.input.SMT.txt",
  "STS.2014.test.input.deft-forum.txt",
  "STS.2014.test.input.deft-news.txt",
  "STS.2014.test.input.headlines.txt",
  "STS.2014.test.input.images.txt",
  "STS.2014.test.input.OnWN.txt",
  "STS.2014.test.input.tweet-news.txt"]
TEST_2015_NAMEFILES=[
  "STS.input.answers-forums.txt",
  "STS.input.answers-students.txt",
  "STS.input.belief.txt",
  "STS.input.headlines.txt",
  "STS.input.images.txt"]

OUTPUT_DIR='output'


In [3]:
def read_data_file(filename):
    GS=[]
    DATA=[]
    with open(filename) as f:
        for line in f:
            line=line.strip()
            bits=line.split()
            #print [x.split(':') for x in bits[1:]]
            line=[ float(x.split(':')[1]) for x in bits[1:]]
            if not np.isfinite(line).all():
                print "problem"
                print line
                continue
            GS.append(float(bits[0]))
            DATA.append(line)
    return GS,np.array(DATA)

def make_data_name(dirname,filename):
    filename=filename.replace(".txt",".dat")
    return os.path.join("..",dirname,filename)

In [4]:
DATA_TRAIN_2015={}
DATA_TEST_2015={}

for filename in TRAIN_2015_NAMEFILES:
    filename_=make_data_name(TRAIN_2015_DATADIR,filename)
    gs,data=read_data_file(filename_)
    DATA_TRAIN_2015[filename]=(gs,data)
    print "Shape para",filename,data.shape

for filename in TEST_2015_NAMEFILES:
    filename_=make_data_name(TEST_2015_DATADIR,filename)
    gs,data=read_data_file(filename_)
    DATA_TEST_2015[filename]=(gs,data)
    print "Shape para",filename,data.shape
    DIMS_DATA=data.shape[1]
    
print "Total train data files 2015:",len(DATA_TRAIN_2015)
print "Total test  data files 2015:",len(DATA_TEST_2015)    

Shape para STS.2012.test.input.MSRpar.txt (750, 16)
Shape para STS.2012.test.input.MSRvid.txt (750, 16)
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999631024565799, 1.0, 1.0, nan, 1.0, 1.0, 0.0]
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999631024565799, 1.0, 1.0, nan, 1.0, 1.0, 0.0]
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999631024565799, 1.0, 1.0, nan, 1.0, 1.0, 0.0]
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999631024565799, 1.0, 1.0, nan, 1.0, 1.0, 0.0]
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999631024565799, 1.0, 1.0, nan, 1.0, 1.0, 0.0]
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999631024565799, 1.0, 1.0, nan, 1.0, 1.0, 0.0]
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999631024565799, 1.0, 1.0, nan, 1.0, 1.0, 0.0]
problem
[1.0, 1.0, 0.9998734907936643, 1.0, 1.0, 1.0, 1.0, 1.0,

In [5]:
X_train_2015=[]
Y_train_2015=[]
X_test_2015=[]
Y_test_2015=[]

first=False
for filename,(gs,data) in DATA_TRAIN_2015.iteritems():
    X_train_2015.append(data)
    Y_train_2015.append(gs)
X_train_2015=np.concatenate(X_train_2015,axis=0)
Y_train_2015=np.concatenate(Y_train_2015,axis=0)


print "Training shape", X_train_2015.shape
print "Training shape", Y_train_2015.shape
    


Training shape (11319, 16)
Training shape (11319,)


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score


X_train_2015=np.nan_to_num(X_train_2015)
rf= RandomForestRegressor()
rf.fit(X_train_2015,Y_train_2015)


for filename,(gs,data) in DATA_TEST_2015.iteritems():
    gs_=rf.predict(data)

    print "===== For",filename
    #print "EVS:", explained_variance_score(gs,gs_)
    #print "MAE:", mean_absolute_error(gs,gs_)
    #print "MSR:", mean_squared_error(gs,gs_)
    #print "mAE:", median_absolute_error(gs,gs_)
    #print "R2 :", r2_score(gs,gs_)
    filename_=os.path.join("..",OUTPUT_DIR,filename)
    testfilename=os.path.join(filename)
    testfilename=testfilename.replace("STS.input","STS.2015.gs")

    fn=open(filename_,'w')
    for pred in gs_: 
        print >> fn, "{0:1.3f}".format(pred)
    fn.close()
    print "PEA:",utils.eval(CMD,os.path.join("..",TEST_2015_DIR,testfilename),filename_)



===== For STS.input.answers-forums.txt
PEA: 0.56314
===== For STS.input.images.txt
PEA: 0.79425
===== For STS.input.answers-students.txt
PEA: 0.6098
===== For STS.input.headlines.txt
PEA: 0.77166
===== For STS.input.belief.txt
PEA: 0.68654
